In [1]:
%matplotlib inline
import os
import urllib.request


In [2]:
import numpy as np
import pandas as pd
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.nn.functional
import torch.optim
from sklearn.metrics import accuracy_score
from torchmetrics.functional import precision_recall
import matplotlib
import matplotlib.pyplot as plt
import Bio
from Bio.Seq import Seq

In [3]:
normal_rna = Seq("GCATGCTGCATGCATGATGCCATGCATGCTGCATGCTGCATG")
tumor_rna =  Seq("GCATGCTGCATGCATGATGCCACGCATGCTGCATGCTGCATG")

print(" 野生型RNA配列: .." + str(normal_rna) + "..")
print("腫瘍特異的蛋白質配列: .." + str(tumor_rna) + "..")

print(" 野生型タンパク質配列: .." + str(normal_rna.translate()) + "..")
print("腫瘍特異的タンパク質配列: .." + str(tumor_rna.translate()) + "..")
"""
vcf ファイル上のすべての変異に対して、
変異ペプチドのリストを網羅的に作成すると同時に、
野生型タンパク質配列と比較し、
野生型タンパク質配列にその断片が含まれないことを確認するという手順を実施して
個々の腫瘍が有する体細胞変異や挿入欠失に基づいた、発現しうる腫瘍特異的ペプチドのリストを作成する。
"""

 野生型RNA配列: ..GCATGCTGCATGCATGATGCCATGCATGCTGCATGCTGCATG..
腫瘍特異的蛋白質配列: ..GCATGCTGCATGCATGATGCCACGCATGCTGCATGCTGCATG..
 野生型タンパク質配列: ..ACCMHDAMHAACCM..
腫瘍特異的タンパク質配列: ..ACCMHDATHAACCM..


'\nvcf ファイル上のすべての変異に対して、\n変異ペプチドのリストを網羅的に作成すると同時に、\n野生型タンパク質配列と比較し、\n野生型タンパク質配列にその断片が含まれないことを確認するという手順を実施して\n個々の腫瘍が有する体細胞変異や挿入欠失に基づいた、発現しうる腫瘍特異的ペプチドのリストを作成する。\n'

In [4]:
data_dir = './pepdata/'
if not os.path.exists(data_dir):
    os.mkdir(data_dir)
    
train_url = 'https://github.com/haoqing12/APPM/raw/master/DATA/train_data/A0301'
test_url = 'https://github.com/haoqing12/APPM/raw/master/DATA/test_data/A0301'


train_save_path = os.path.join(data_dir, 'train_A0301.csv')
test_save_path = os.path.join(data_dir, 'test_A0301.csv')

urllib.request.urlretrieve(train_url, train_save_path)
urllib.request.urlretrieve(test_url, test_save_path)

pd.read_csv(train_save_path, nrows=5).head



<bound method NDFrame.head of       Peptide      HLA  BindingCategory
0   GTFGGLGSK  A*03:01                1
1   TTQDPLSNK  A*03:01                1
2  TVAGGAWTYK  A*03:01                1
3  FVYDHVFAEK  A*03:01                1
4   KVTEWQQTY  A*03:01                1>

In [5]:
# 前処理
class PepTransform():
    allSequences = 'ACEDGFIHKMLNQPSRTWVYZ'
    char2int = dict((c, i) for i, c in enumerate(allSequences)) # Add index to amino seq
    
    def peptideOneHotMap(self, peptide):
        peptide_integer = [self.char2int[char] for char in peptide] # convert peptide(str) to int 
        peptide_onehot = list() # peptide seq in Onehot vector format
        for value in peptide_integer:
            letter = [0 for _ in range(len(self.allSequences))]
            
            letter[value] = 1
            peptide_onehot.append(letter)
            
        return np.asarray(peptide_onehot)
        
    def __init__(self):
        '''
        initialize instance variable
        '''
        
    def __call__(self, peptide):
        '''
        与えられたpeptide配列に対応するOneHotベクトルのリストを返す。
        '''
        return self.peptideOneHotMap(peptide)
        

In [6]:
# ネオ抗原となりうるペプチド長は8~11merと言われるので、11文字に固定して、足りない部分はZで埋める
def complementPeptides(peptides_series:np.ndarray):
    peptides_list = peptides_series.tolist() # tolistはndarrayのメソッド
    
    for i in range(len(peptides_list)):
        if len(peptides_list[i]) < 11:
            peptides_list[i] = peptides_list[i] + 'Z' * (11 - len(peptides_list))
        else:
            peptides_list[i] = peptides_list[i][:11]
        print("Generated Peptide is: " + peptides_list[i])
    return peptides_list

peptides_list = ['ACCMHDACCMHDAAA', 'ACCMHDATH']
for i in range(len(peptides_list)):
    if len(peptides_list[i]) < 11:
        peptides_list[i] = peptides_list[i] + 'Z' * (11 -len(peptides_list[i]))
    else:
        peptides_list[i] = peptides_list[i][:11]
        
    print("Generated Pepetide is: " + peptides_list[i])



Generated Pepetide is: ACCMHDACCMH
Generated Pepetide is: ACCMHDATHZZ


In [7]:
class MakeDataset(data.Dataset):
    def __init__(self, save_path, transform=None):
        super().__init__()
        '''
        Attributes:
            save_path: ペプチドのリストを有するテキストファイル
            transform: 前処理クラスのインスタンス
        Returns:
            X: OneHotベクトルに変換したtorch.Tensor型
        '''
        df = pd.read_csv(save_path, header=0)
        df = df[df['Peptide'].str.contains('X') == False] # 不明なペプチドが含まれているPeptide配列を削除
        df = df[df['Peptide'].str.contains('B') == False] # 不明なペプチドが含まれているPeptide配列を削除
        df = df[df['Peptide'].str.contains('U') == False] # 不明なペプチドが含まれているPeptide配列を削除
        
        print('{0}のファイル：'.format(save_path))
        print('#Original Negative data is: ' + str(df.loc[df['BindingCategory'] == 0].shape))
        print('#Original Positive data is: ' + str(df.loc[df['BindingCategory'] == 1].shape) + '\n')
        
        # 学習時だけPositiveサンプルのオーバーサンプリングを行う。
        if "train" in save_path:
            df_0 = df.loc[df['BindingCategory'] == 0]
            df_1 = df.loc[df['BindingCategory'] == 1]
            df = pd.concat([df_0, df_1, df_1], axis=0)
            print('オーバーサンプリング後：')
            print('#Original Negative data is: ' + str(df.loc[df['BindingCategory'] == 0].shape))
            
        df = df.sample(frac=1).reset_index() # 100%行と列をサンプリングして抽出。インデックスを初期化して振りあて
        
        self.peptides = complementPeptides(df['Peptide'])
        self.y = df['BindingCategory']
        self.transform = transform # 前処理クラスのインスタンス
        
    def __len__(self):
        '''
        len(obj)で実行されたときにコールされる関数
        '''
        return len(self.peptides)
    
    def __getitem__(self, index):
        '''Datasetクラスの__getitem__をオーバーライドする。'''
        # index番目のペプチドを取得
        peptide_transformed = self.transform(self.peptides[index])
        
        X = torch.from_numpy(peptide_transformed.astype(np.float32)).clone()
        Y = torch.tensor(self.y[index], dtype=torch.long).clone()
        return torch.reshape(X, (1, X.shape[0], X.shape[1])), Y
                    
        
        
        
        
        

In [16]:
batch_size = 32

# MakeDatasetで訓練データと正解ラベルを取得
train_dataset = MakeDataset(
    save_path = train_save_path,
    transform = PepTransform()
)




# MakeDatasetで検証データと正解ラベルを取得
test_dataset = MakeDataset(
    save_path = test_save_path,
    transform = PepTransform()
)

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=False
)


test_dataloader = torch.utils.data.DataLoader(
    test_dataset,  batch_size=batch_size, shuffle=False
)



# データローダーが返すミニバッチの先頭データの形状を出力して確認する。
"""
for x, t in train_dataloader:
    print(x.shape)
    print(t.shape)
    break

for (x, t) in test_dataloader:
    print(x.shape)
    print(t.shape)
    break
"""





./pepdata/train_A0301.csvのファイル：
#Original Negative data is: (73112, 3)
#Original Positive data is: (4419, 3)

オーバーサンプリング後：
#Original Negative data is: (73112, 3)
Generated Peptide is: LREGGSAVD
Generated Peptide is: GTGLDELFV
Generated Peptide is: HGQIDSMEQ
Generated Peptide is: VFAARGPAPAP
Generated Peptide is: IEFTTASTL
Generated Peptide is: ALVSGKSTAK
Generated Peptide is: DLAKGKKKYF
Generated Peptide is: RYICAECGKA
Generated Peptide is: EIYKKGQSK
Generated Peptide is: VSVFEPKAS
Generated Peptide is: SSRVCSLLVFA
Generated Peptide is: KKVWSALPYT
Generated Peptide is: KGIPVYPYP
Generated Peptide is: KIKQVDSVLK
Generated Peptide is: RSMTDPQNM
Generated Peptide is: ENNLHADFAK
Generated Peptide is: NCTQPAVVTKD
Generated Peptide is: QNAQEDEYIQ
Generated Peptide is: QETDFSEAS
Generated Peptide is: IWILMPFVKTC
Generated Peptide is: SLFGPRSLQK
Generated Peptide is: ELGRGVRKLGC
Generated Peptide is: PMPPPPLGL
Generated Peptide is: ESSLTEAYEKW
Generated Peptide is: IFQGILFSS
Generated Peptide 

'\nfor x, t in train_dataloader:\n    print(x.shape)\n    print(t.shape)\n    break\n\nfor (x, t) in test_dataloader:\n    print(x.shape)\n    print(t.shape)\n    break\n'